In [1]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, make_scorer, roc_curve, auc
from sklearn.pipeline import Pipeline
from skopt.space import Real, Categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

Utiliza optimización bayesiana para intentar encontrar el mínimo global de la siguiente función:


donde las 3 variables están acotadas en [0, 1]. Crea 5 muestras iniciales e itera 15 veces para optimizar.

 

Explica por qué optimización bayesiana es una elección buena para este problema en lugar de GridSearch.

In [87]:
np.random.seed(10)

def f(x,y,z):
    return ((6*x - 2)**2) * (np.sin(12*x-4)) + ((6*y - 2)**2) * np.cos(12*y-4) + ((6*z-2)*np.sin(12*z-4))
 
X_f = np.random.uniform(0, 1, 5).reshape([-1, 1])
Y_f = np.random.uniform(0, 1, 5).reshape([-1, 1])
Z_f = np.random.uniform(0, 1, 5).reshape([-1, 1])

a = np.hstack([X_f, Y_f, Z_f])

In [88]:
kernel = 1.0*RBF(length_scale = 1)
gp = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer=10)
y = f(X_f,Y_f,Z_f)
gp.fit(a, y)

GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1),
                         n_restarts_optimizer=10)

In [89]:
x_x = np.linspace(0, 1, 100).reshape(-1, 1)
x_y = np.linspace(0, 1, 100).reshape(-1, 1)
x_z = np.linspace(0, 1, 100).reshape(-1, 1)

x_v = np.hstack([x_x, x_y, x_z])

y_pred, std_pred = gp.predict(x_v, return_std=True)

In [90]:
x_max_pred, std = gp.predict(x_v, return_std = True)

y_upper = y_pred + 1.96*std
y_lower = y_pred - 1.96*std

In [91]:
indice_pred = np.argmax(y_pred)
x_max_pred = x_v[indice_pred][0]
#y_max = f(x_max_pred)
x_max_pred

0.16161616161616163

In [92]:
np.random.seed(14)

# Bucle de 15 iteraciones
for i in range(15):
    gp.fit(a, y)
    y_pred, std = gp.predict(x_v, return_std=True)

    # Encuentra el punto con mayor incertidumbre
    indice_pred = np.argmax(y_pred)
    x_max_pred = x_v[indice_pred][0]
    y_max = f(x_max_pred, x_max_pred, x_max_pred)

    # Agrega el nuevo punto
    X = np.vstack([a, x_max_pred])
    y = np.vstack((y.reshape(-1, 1), y_max))

# Predicción final
gp.fit(a, y)
y_pred_final, std_final = gp.predict(x_v, return_std=True)

# Intervalo de confianza
y_upper = y_pred_final + 1.96*std_final
y_lower = y_pred_final - 1.96*std_final

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 1

El dataset adidas contiene información acerca de productos y la calificación que se les dio por usuarios. Utiliza regresión logística para conseguir un modelo que nos ayude a decidir si un producto será clasificado como bueno (average_rating>=4.3).

Encuentra la significancia estadística de los factores utilizados.

Cross-validation a utilizar: Train-test.

Métrico a utilizar: AUC. Explica el significado el score que arroje tu modelo.

In [68]:
data = pd.read_csv("adidas.csv")

In [81]:
df = data.dropna().drop_duplicates()
df.head(2)

,url,name,sku,selling_price,original_price,currency,availability,color,category,source,source_website,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at
15,https://www.adidas.com/us/essentials-loose-log...,Essentials Loose Logo Tank Top,H07758,20,$25,USD,InStock,Purple,Clothing,adidas United States,https://www.adidas.com,Women/Clothing,What's on the agenda? Whether it's a quick lun...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.8,116,2021-10-23 17:50:19.626607
16,https://www.adidas.com/us/essentials-loose-log...,Essentials Loose Logo Tank Top,H07757,20,$25,USD,InStock,Pink,Clothing,adidas United States,https://www.adidas.com,Women/Clothing,What's on the agenda? Whether it's a quick lun...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.8,116,2021-10-23 17:50:19.814818


In [83]:
X = df[["selling_price", "reviews_count"]]
y = df["average_rating"]

In [84]:
log_clf = make_pipeline(StandardScaler(),
                        LogisticRegression(penalty='l2', C=1, solver='lbfgs', max_iter=1000, random_state=42))

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
log_clf.fit(X_train, y_train)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.